In [1]:
from urllib.parse import quote_plus    # 한글 텍스트를 퍼센트 인코딩으로 변환
from selenium import webdriver    # 라이브러리에서 사용하는 모듈만 호출
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait   # 해당 태그를 기다림
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException    # 태그가 없는 예외 처리
import time
import pandas as pd
import requests
from selenium import webdriver

In [2]:
chromedriver = '/Users/pym/Desktop/dsstack-3/shared/notebooks/3_web-scraping/chromedriver'
driver = webdriver.Chrome(chromedriver)
#driver.maximize_window() # 최대화

In [5]:
driver.get('https://some.co.kr/')
time.sleep(2)

In [4]:
#팝업 광고에서 오늘하루동안 보지 않기 클릭
add_path ='#sometrend_update_popup > section > div.textarea_area > div > label'
add_1 = driver.find_element_by_css_selector(add_path)
add_1.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#sometrend_update_popup > section > div.textarea_area > div > label"}
  (Session info: chrome=85.0.4183.102)


In [ ]:
xpath = '/html/body/div[1]/div/div[2]/div[2]/div[2]/input' #검색창 xpath
clickpath = 'searchKeywordClick' #검색창에 입력후 검색버튼 
periodpath = '#wrap > div.mo_analysis_top_wrap > section > div > div > section.search_section.section_btn > div.label_box > label:nth-child(4)'#기간을 한달로 설정하기 위한 버튼

In [ ]:
sumt = driver.find_element_by_xpath(xpath)
sumt_submitbutton = driver.find_element_by_id(clickpath)

#광고가 뜨기때문에 광고제거
try:
    sumt_add = driver.find_element_by_xpath(addxpath)
    sumt_add.click()
except:
    pass
time.sleep(2)

sumt.send_keys('원빈')
sumt_submitbutton.click()
time.sleep(2)

In [ ]:
driver.maximize_window() # 최대화
time.sleep(2)
sumt_period = driver.find_element_by_css_selector(periodpath) #1달 기간 클릭
sumt_period.click()
time.sleep(2)

In [ ]:
#sensbutton_path = '#wrap > section.section.sensitive_wrap.mgTop_50 > div > div > h3 > a.accordion_btn.received.wide'
#감성어 변화 펼치기 클릭
sensbutton_xpath = '/html/body/div[2]/section[3]/div/div/h3/a[1]'
sensitive_button = driver.find_element_by_xpath(sensbutton_xpath)
sensitive_button.click()

In [380]:
driver.maximize_window() # 최대화

In [11]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import re

In [12]:
video = driver.page_source
soup = BeautifulSoup(video, 'html.parser')

In [13]:
scrap_1 = soup.find_all('td',class_='ta_center')

In [14]:
#scrap_1 #순위랑 분류 포함되어있음

In [15]:
scrap_2 = soup.find_all(class_='keywordText')

In [16]:
#scrap_2 #감정어 포함되어 있음

In [17]:
scrap_3 = soup.find_all('td', class_='numberFont')

In [388]:
#scrap_3 #해당 감정어 누적 카운트

In [18]:
#몇주차인지
date = soup.find_all(class_='week-date')

date_lst = []
for i in range(5):
    a = date[i].get_text()
    date_lst.append(a)

date_a = date_lst * 10
date_a = sorted(date_a) 

In [19]:
#상세날짜
detail_date = soup.find_all(class_='detail-date')

detail_date_lst = []
for i in range(5):
    b = detail_date[i].get_text()
    detail_date_lst.append(b)
    
detail = detail_date_lst * 10
detail = sorted(detail)

In [44]:
pattern = re.compile('[가-힣0-9]+')
scrap_first = pattern.findall(str(scrap_1))
ranking= scrap_first[0::2] #리스트에서 홀수 추출(랭킹)
classification=scrap_first[1::2] #리스트에서 짝수 추출(분류)

In [45]:
sensitive = []
for i in range(50):
    a = scrap_2[i].get_text()
    sensitive.append(a)

In [46]:
pattern3 = re.compile('[0-9+,]+')
count = pattern3.findall(str(scrap_3))
count = count[0::2]

In [47]:
print(len(ranking))
print(len(classification))
print(len(sensitive))
print(len(count))
print(len(date_a))
print(len(detail))

50
50
50
50
50
50


In [48]:
df = pd.DataFrame({'ranking':ranking, 'classification':classification, 
                'sensitive':sensitive, 'count':count, 'date':date_a, 'detail_date':detail})

In [49]:
df

,ranking,classification,sensitive,count,date,detail_date
0,1,긍정,공감,356,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
1,2,긍정,잘생기다,20,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
2,3,중립,유명한,18,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
3,4,중립,웃기다,16,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
4,5,중립,닮다,12,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
5,6,긍정,좋다,11,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
6,7,긍정,좋아하다,10,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
7,8,긍정,잘하다,5,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
8,9,부정,못생기다,5,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)
9,10,긍정,좋은,4,2020년 08월 2주차,(2020.08.12 ~ 2020.08.16)


In [421]:
soup.find_all('tspan')

[<tspan>100</tspan>,
 <tspan>-200</tspan>,
 <tspan>-100</tspan>,
 <tspan>0</tspan>,
 <tspan>100</tspan>,
 <tspan>200</tspan>,
 <tspan>300</tspan>,
 <tspan>400</tspan>,
 <tspan>500</tspan>,
 <tspan>600</tspan>,
 <tspan>700</tspan>,
 <tspan>800</tspan>,
 <tspan>L</tspan>,
 <tspan>2020.08.12</tspan>,
 <tspan>2020.08.16</tspan>,
 <tspan>2020.08.20</tspan>,
 <tspan>2020.08.24</tspan>,
 <tspan>2020.08.28</tspan>,
 <tspan>2020.09.01</tspan>,
 <tspan>2020.09.05</tspan>,
 <tspan>2020.09.09</tspan>,
 <tspan>100%</tspan>]